In [69]:
import torch
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

# Проверка наличия GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Аугментация данных и преобразования
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Путь к корневой директории с данными
data_dir = './'

# Создание структуры данных с поддиректориями "train" и "val"
# Папки "Black" и "Grizli" должны находиться внутри "train" и "val"
train_dir = f'{data_dir}/train'
val_dir = f'{data_dir}/val'

# Загрузка данных с помощью ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(root=train_dir, transform=data_transforms['train']),
    'val': datasets.ImageFolder(root=val_dir, transform=data_transforms['val']),
}

# Создание DataLoader'ов
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=True, num_workers=4),
}

# Размеры выборок
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

print(f'Классы: {class_names}')

Классы: ['Вадим', 'Инна']


In [70]:
# Загрузка предобученной модели ResNet152
model_ft = models.resnet152(pretrained=True)

# Замораживаем все параметры модели, кроме последнего слоя
for param in model_ft.parameters():
    param.requires_grad = False

# Замена финального полносвязного слоя на слой с 2 выходами (для двух классов)
num_ftrs = model_ft.fc.in_features
model_ft.fc = torch.nn.Linear(num_ftrs, len(class_names))

# Перемещаем модель на устройство (CPU/GPU)
model_ft = model_ft.to(device)

In [71]:
criterion = nn.CrossEntropyLoss()

# Оптимизатор будет обновлять только параметры последнего слоя
optimizer_ft = optim.Adam(model_ft.fc.parameters(), lr=0.001)

In [72]:
def train_model(model, criterion, optimizer, num_epochs=25):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Каждая эпоха имеет фазу обучения и валидации
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Обнуление градиентов
                optimizer.zero_grad()

                # Прямой проход
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Обратный проход и оптимизация только в фазе обучения
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Статистика
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Сохранение лучшей модели
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    print(f'Best val Acc: {best_acc:.4f}')

    # Загрузка лучших весов модели
    model.load_state_dict(best_model_wts)
    return model

# Обучение модели
model_ft = train_model(model_ft, criterion, optimizer_ft, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.7592 Acc: 0.5128
val Loss: 0.7046 Acc: 0.5000

Epoch 1/24
----------
train Loss: 0.5137 Acc: 0.7564
val Loss: 0.5123 Acc: 0.8333

Epoch 2/24
----------
train Loss: 0.4398 Acc: 0.7821
val Loss: 0.4383 Acc: 0.8542

Epoch 3/24
----------
train Loss: 0.3602 Acc: 0.8846
val Loss: 0.5009 Acc: 0.6458

Epoch 4/24
----------
train Loss: 0.2868 Acc: 0.9295
val Loss: 0.3960 Acc: 0.7917

Epoch 5/24
----------
train Loss: 0.2070 Acc: 0.9808
val Loss: 0.2899 Acc: 0.9583

Epoch 6/24
----------
train Loss: 0.1573 Acc: 0.9808
val Loss: 0.2770 Acc: 0.9375

Epoch 7/24
----------
train Loss: 0.1317 Acc: 0.9936
val Loss: 0.2394 Acc: 0.9583

Epoch 8/24
----------
train Loss: 0.1601 Acc: 0.9679
val Loss: 0.2327 Acc: 0.9375

Epoch 9/24
----------
train Loss: 0.1445 Acc: 0.9679
val Loss: 0.2101 Acc: 0.9792

Epoch 10/24
----------
train Loss: 0.1110 Acc: 0.9936
val Loss: 0.1953 Acc: 0.9792

Epoch 11/24
----------
train Loss: 0.1000 Acc: 0.9872
val Loss: 0.1818 Acc: 0.9792

Ep

In [73]:
def evaluate_model(model):
    model.eval()
    corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            corrects += torch.sum(preds == labels.data)

    acc = corrects.double() / dataset_sizes['val']
    print(f'Validation Accuracy: {acc:.4f}')

# Оценка модели
evaluate_model(model_ft)

Validation Accuracy: 0.9792


In [74]:
torch.save(model_ft.state_dict(), 'human_classifier_resnet152.pth')

In [84]:
from PIL import Image
import torch.nn.functional as F

def predict_image(image_path, model):
    model.eval()

    # Преобразование изображения
    image = Image.open(image_path)
    transform = data_transforms['val']
    image = transform(image).unsqueeze(0).to(device)

    # Предсказание
    with torch.no_grad():
        output = model(image)
        probabilities = F.softmax(output, dim=1)
        _, pred = torch.max(probabilities, 1)
        prob = probabilities[0][pred].item()

    return class_names[pred], prob

# Пример использования
image_path = 'B14dos3.jpg'
prediction, probability = predict_image(image_path, model_ft)
print(f'Я распознал фото на нём: {prediction} с вероятностью {probability * 100:.1f}%)')

Я распознал фото на нём: Вадим с вероятностью 70.3%)
